In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.chat_loaders.gmail import GMailLoader
from langchain_chroma import Chroma

In [4]:
load_dotenv(override=True)
api_key = os.environ["OPENAI_API_KEY"]
openai = OpenAI()

In [ ]:
from pathlib import Path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from IPython.display import JSON, display

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
TOKEN_PATH = Path("token.json")
CLIENT_SECRETS = "client_secret.json"

def get_creds():
    creds = None
    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)
    # If there are no (valid) credentials, let user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS, SCOPES)
            creds = flow.run_local_server(port=0)  # opens browser for consent
        # Save the credentials for next run
        with open(TOKEN_PATH, "w") as f:
            f.write(creds.to_json())
    return creds

def list_messages(service, user_id="me", max_results=10):
    res = service.users().messages().list(userId=user_id, maxResults=max_results).execute()
    messages = res.get("messages", [])
    return messages

def get_message(service, msg_id, user_id="me", format="full"):
    return service.users().messages().get(userId=user_id, id=msg_id, format=format).execute()

def main():
    creds = get_creds()
    service = build("gmail", "v1", credentials=creds)
    msgs = list_messages(service, max_results=10)
    # print(f"Found {len(msgs)} messages (IDs):")
    # for m in msgs:
    #     print(" -", m["id"])

    # Fetch and print snippet of first message (if any)
    if msgs:
        full = get_message(service, msgs[0]`["id"], format="full")
        snippet = full.get("snippet", "")
        # print("\nFirst message snippet:")
        # print(snippet)
        display(snippet)

In [23]:
main()

'You allowed PersonalFlow access to some of your Google Account data soroushav85@gmail.com If you didn&#39;t allow PersonalFlow access to some of your Google Account data, someone else may be trying to'